In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [3]:
df_sonnet = df[df.Form == 'Sonnet']

In [4]:
poems = df_sonnet['Poem']
poems.head(20)

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
5     Then let not winter's ragged hand deface,\r\n ...
6     Lo! in the orient when the gracious light\r\n ...
7     Music to hear, why hear'st thou music sadly?\r...
8     Is it for fear to wet a widow's eye,\r\n  That...
9     For shame! deny that thou bear'st love to any,...
10    As fast as thou shalt wane, so fast thou grow'...
11    When I do count the clock that tells the time,...
12    O! that you were your self; but, love you are\...
13    Not from the stars do I my judgement pluck;\r\...
14    When I consider every thing that grows\r\n  Ho...
15    But wherefore do not you a mightier way\r\n  M...
16    Who will believe my verse in time to come,\r\n...
17    Shall I compare thee to a summer's day?\r\

In [5]:
text = ''
for poem in poems:
    text += str(poem) + '\n'
len(text)

147650

In [6]:
print(text)

From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou, contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thyself thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
        Pity the world, or else this glutton be,
        To eat the world's due, by the grave and thee.
When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tatter'd weed of small worth held:
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say, within thine own deep sunken eyes,
  Were an all-eating shame, and thr

In [7]:
maxlen = 30                                                            
step = 3                                                               

sentences = []                                                         

next_chars = []                                                        

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))                                        
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)      
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, char in enumerate(sentence):                                
        x[i, t, char_indices[char]] = 1                                
    y[i, char_indices[next_chars[i]]] = 1 

Number of sequences: 49207
Unique characters: 70
Vectorization...


In [8]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [9]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
import random
import sys

for epoch in range(1, 21):                                          
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(text) - maxlen - 1)         
    generated_text = text[start_index: start_index + maxlen]        
    print('--- Generating with seed: "' + generated_text + '"')     
    for temperature in [0.2, 0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(chars)))             
            for t, char in enumerate(generated_text):               
                sampled[0, t, char_indices[char]] = 1.              

            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temperature)                 
            next_char = chars[next_index]                           

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
49207/49207 [==============================] - 55s 1ms/step - loss: 2.2742
--- Generating with seed: "ial hue.
So when my tongue wo"
------ temperature: 0.2
ial hue.
So when my tongue wore the will the wail the wall the that the wall the wall the wail the wall the wall the wail the wail the wail the wall the wall the wall the wall the wail the wail the wail the wail the wall whe the wail the wail the wall the wail what the wail the wall the wall the wall the wall the wail the wail the wail the wart the pall the wail the wail the wall the wail the wall the hand the wart the wall t------ temperature: 0.5
l the hand the wart the wall the waild betond the wart thy waild and woll in the stait the caold as thee that all of hand the all the sould and the uad thou bul ment,
  That the wall the hand the worl dest that the nuth and the wall the wart sand,
        And the thine that sull the wall and this han on the all and and a deas the wall the bat the wall in the waice yet w

  And with thou all the thing the ward a sing and look with thee thee thee thee thee the stard and thee thee thee thee are that me his stare
  To love that the warld the ward with thou more s------ temperature: 0.5
arld the ward with thou more starn thee and all wrong and beauty she thee of the like a still that it thought,
Which have look belovëd, as my self-love thee,
  To beauty may make of thy beauty to thing,
  And wat that have mane, bet me like,
  And excearing your see poor that hing as your sound a day,
  When her look in thy seatur to heart
If shall faul the bure for thee that thee thine own see and beauty better with fai------ temperature: 1.0
see and beauty better with fait inser fateing huil,
  And love, thy boodant entoveroof such aftercep.  yek st  their stare
’se may majoke! as, lake and cooken one itsail,
And comit thy bart that being stapertst fanted,
  To hopp merest of my away, thy selfe of your saye
  know mine own mine owness's conaition dreath,
Yel but love thou 

Which in her poet wat thou hast thy light,
  And then thee but the can as to see mind,
  And who changers beauty dosposs and make thee,
        Then my pecrigg shall slave t------ temperature: 1.0
 Then my pecrigg shall slave truth.
  noves for sincressing blass and sinccued:
              So thou shall can my worth seemy when him.
  On sweet surlevast hall loves mind,
  
  And ever stringing gorsed my danck,
  And thinks all my dear, but shay of naught have say it,'he your self.
  O! nace what to seep what so frours, and yut toillly twast-mip'dleOs tome;
  For thinking mind it pervose thee on outs to biTa,
  S------ temperature: 1.2
ose thee on outs to biTa,
  Since I freat thou hustear hussy selod! let deasjud;
  And moveerse worke thus in funted decope,
  ThaFce these mantionly self do rimzlet,
  By tomder only deventie a hathes.
So my beauty looty! mene, neverlucl? with a fruind!
  in worthin and gonds xscirit,
        Then with uchin and erjive away.
  Ib as way formarts here chue

  I love, by addeit mine reich him leats,
  As sweet preasunce upon the eyer be.
ghase, to the cho
As thy mind and core, I enwrall.
  Ye ching in a best that eyes.
hapry thoo hall I s------ temperature: 1.2
that eyes.
hapry thoo hall I some amrilgt;
  And thoue livily conternimy O frow:
  So, in the words, that drops, that every blay
To liftange in my eprart, to Live my on thyseam.h. Thin thou By pearth, or forthing conforse.
Is, lov's death's chee, but now play cold?
  Tife you amoof pould not theight,
She bare, each eHnen myself to ly;
  Which worthus showmy wrailed me,
  Thysing loduy  own selver's iff,
etness faith theepoch 14
Epoch 1/1
49207/49207 [==============================] - 53s 1ms/step - loss: 1.1819
--- Generating with seed: "ing, he went wilful-slow,
   "
------ temperature: 0.2
ing, he went wilful-slow,
        Ther true do thy canst do the compance,
  That thou art self-lives thee the come
And the world and the bear a call,
        Then thou hast thou hast thou hast t

  Though hus wollied that but in digse did :
edMad Oloth hasthing for my loev'st his crifes put shall greas my mard
  To love thee, Love's longant do treasure love, warthers joy.
mance my BelyBs samepoch 18
Epoch 1/1
49207/49207 [==============================] - 50s 1ms/step - loss: 1.1323
--- Generating with seed: "e to write for me,
  Under th"
------ temperature: 0.2
e to write for me,
  Under that this still thou thy love and my love,
  And in heavenly thou art fair to me,
  And beauty that then thy power constance.
So not summer is in thy dear hate.
So I love that so me thou shouthuness of the stare,
  And in heavenly the starp'd from thee,
  And beauty doo the beauty that thine her frell of the stare of the stare.
If thy love shall strong starde.
That thou art frow as the stare of ------ temperature: 0.5
thou art frow as the stare of love,
  Thou hast thy before in me sweet sun thee doth like to be the dast the self since seem
  To self-  ill seem to men hath me,
  The self with